In [39]:
class Expectations:
    def __init__(self, manifestvars, latentvars, interceptload, slopeload):
        #add the default information into the class
        import sympy as sy
        import networkx as nx
        self.manifestvars = manifestvars
        self.latentvars = latentvars
        self.interceptload = interceptload
        self.slopeload = slopeload
    
    def create_graph_from_data(self, adjlist):
        # adjlist in the format of ['lvi mv1 mv2 mv3 mv4', 'lvs mv1 mv2 mv3 mv4', 'constant lvi lvs']
        #parse the directed graph adjacency list
        input_graph = nx.parse_adjlist(adjlist, create_using=nx.DiGraph())
        # the next important part is add the factor loadings to the graph. 
        loading_index = 0
        variable_index = 0
        for latentvar in latentvars:
            loading_index = 0
            for node_i in input_graph.successors(latentvar):
                if variable_index == 0 :
                    input_graph[latentvar][node_i]['factor_loading'] = interceptload[loading_index]
                    loading_index += 1
                else:
                    input_graph[latentvar][node_i]['factor_loading'] = slopeload[loading_index]
                    loading_index += 1
                variable_index += 1
        
            
    def read_data_Outlist(self):#manifestvars, latentvars, interceptload, slopeload):
        # Each variabe name should be in a separate str in a list
        # e.g., ["lvi", "lvs"]
        # manifest variables measured overtime should be in chronological order
        # the input is the  
        adjlist = []
        # easier to call the minfest variables squares at this point
        squares = " ".join(manifestvars)
        for variable in latentvars:
            line = variable + " " + squares
            adjlist.append(line)
        constants = "constant " + " ".join(latentvars)
        adjlist.append(constants)
        # output is the adjacency list to be used in the graph drawing
        return adjlist
    
    def create_symbols(self):
        # the function to create the symbolic alegbra symbols
        # this will use a dictionary in which to store the sybmols that will be used
        import sympy as sy
        matrix_symbols = {}
        for var in latentvars:
            var_name = 'Var(%s)' % var
            mew_name = 'Mean(%s)' % var
            matrix_symbols.update( {'Variance of %s' %(var):sy.Symbol(var_name), 'Mean of %s' %(var):sy.Symbol(mew_name)} )
        matrix_symbols.update( {'Error Variance' :sy.Symbol('Var(error)')})
        cov_name = 'Cov(%s & %s)' %(latentvars[0], latentvars[1])
        matrix_symbols.update( {'Covariance' :sy.Symbol(cov_name)})
        # output is the dictionary that will be used by the matrix creation functions
        return(matrix_symbols)
        
    def create_matrix_f(self, matrix_symbols):
        import sympy as sy
        # input are the lists of manifest variables and latent variables
        # this function assumes that the user wants the means as well as the covariance matrix
        manifestvar_mat = sy.eye(len(manifestvars) + 1)
        latentvar_mat = sy.zeros(len(manifestvars) + 1, len(latentvars))
        matrix_f = manifestvar_mat.col_insert((len(manifestvar_mat)-1), latentvar_mat)
        
        return matrix_f
        
    def create_matrix_a(self, matrix_symbols):
        import sympy as sy
        # this is the matrix that contains the information for the intercept and slope loadings
        matrix_a = sy.zeros(len(manifestvars) + len(latentvars) + 1)
        #zero_matrix
        index = 0
        for loading in enumerate(interceptload):
            matrix_a[index, -2] = interceptload[index]
            matrix_a[index, -1] = slopeload[index]
            index += 1
        matrix_a[-1, len(manifestvars)] = matrix_symbols.get('Mean of lvs')
        matrix_a[-2, len(manifestvars)] = matrix_symbols.get('Mean of lvi')
        
        return matrix_a
    
    def create_matrix_s(self, matrix_symbols):
        # the +1 in the first line is to account for the constant
        # first create the zero matrix, then fill with the appropriate symbols
        import sympy as sy
        matrix_s = sy.zeros(len(manifestvars) + len(latentvars) + 1)
        count_index = 0
        for var in enumerate(manifestvars):
            matrix_s[count_index, count_index] = matrix_symbols.get("Error Variance")
            count_index += 1
        matrix_s[len(manifestvars), len(manifestvars)] = 1
        matrix_s[-2, -2] = matrix_symbols.get("Variance of lvi")
        matrix_s[-1, -1] = matrix_symbols.get("Variance of lvs")
        matrix_s[-1, -2] = matrix_symbols.get("Covariance")
        matrix_s[-2, -1] = matrix_symbols.get("Covariance")
        
        return matrix_s
     
    def calculate_expectations(matrix_f, matrix_a, matrix_s):
        # this function will take the three matrices and calculate the expectations for the covariance matrix and expected means
        import sympy as sy
        iden = sy.eye(6)
        k = (iden - matrix_a)**-1
        matrix_expect = matrix_f * k * matrix_s * matrix_k.T * matrix_f.T
        # the output is a matrix of 
        return matrix_expect

In [31]:
latentvars = ["lvi", "lvs"]
manifestvars = ["mv1", "mv2", "mv3", "mv4", "mv5", "mv6"]
slopeload = [1,2,3,4,5,6]
interceptload = [1,1,1,1]    

In [41]:
h = Expectations(manifestvars, latentvars, interceptload, slopeload)

In [38]:
h.create_symbols()

{'Variance of lvi': Var(lvi),
 'Mean of lvi': Mean(lvi),
 'Variance of lvs': Var(lvs),
 'Mean of lvs': Mean(lvs),
 'Error Variance': Var(error),
 'Covariance': Cov(lvi & lvs)}

In [43]:
matrixsym = h.create_symbols()
a=h.create_matrix_a(matrixsym)
f=h.create_matrix_f(matrixsym)
s=h.create_matrix_s(matrixsym)

Expectations.calculate_expectations(f, a, s)

ShapeError: Matrix size mismatch: (6, 6) + (9, 9)